In [1]:
from langchain_core.output_parsers import JsonOutputParser

from llms import model, ollama_model
from prompts import query_refinement, image_query_extraction
from langchain_core.prompts import ChatPromptTemplate

C:\Users\GIGABYTE\PycharmProjects\VectorsHackthon\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def refine(query):
    prompt = ChatPromptTemplate.from_template(query_refinement)
    chain = prompt | model
    try:
        answer = chain.invoke({"query": query})
        return answer.content
    except Exception as e:
        return {"error": f"Failed to parse: {str(e)}", "raw_query": query}

In [3]:
print(refine("cheap headphones"))

{"semantic_query": "cheap headphones", "filters": { "max_price": 75, "category": "headphones" }, "financial_priority": "low_total_price"}


In [15]:
import base64
from langchain_core.messages import HumanMessage
def describe_image(image_path: str):
    with open(image_path, "rb") as image_file:
        image_data = base64.b64encode(image_file.read()).decode("utf-8")


    message = HumanMessage(
        content=[
            {"type": "text", "text": image_query_extraction},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
            },
        ]
    )

    try:
        response = model.invoke([message])
        return response.content
    except Exception as e:
        return f"Error describing image: {str(e)}"

In [ ]:
print(describe_image("camera.jpg"))

In [4]:
from Hybrid_Search import HybridSearcher
hybrid_searcher = HybridSearcher(collection_name="products")
print(hybrid_searcher.search(text="leather jacket for men"))

[{'actual_price': 35.19, 'discounted_price': 29.99, 'category': "Men's Character Shop", 'rating': 5.0, 'image_url': '"https://i5.walmartimages.com/seo/Men-Sherpa-Lined-Hooded-Jacket-Men-s-Flannel-Plaid-Shirts-Fleece-Fuzzy-Button-Down-Long-Sleeve-Winter-Thermal-Hoodies-Jackets-with-Pockets_dc9e9fe7-4994-4665-8ef5-705f11685c36.e015b9e817f57c0cc85c2bb14dc7746b.jpeg"', 'product_url': 'https://www.walmart.com/ip/Men-Sherpa-Lined-Hooded-Jacket-Men-s-Flannel-Plaid-Shirts-Fleece-Fuzzy-Button-Down-Long-Sleeve-Winter-Thermal-Hoodies-Jackets-with-Pockets/5209344136'}, {'actual_price': 36.95, 'discounted_price': 29.39, 'category': "Men's Character Shop", 'rating': 5.0, 'image_url': '"https://i5.walmartimages.com/seo/Men-Casual-Sherpa-Fleece-Lined-Plaid-Flannel-Shirts-Jackets-Men-s-Long-Sleeve-Plush-Thicken-Hooded-Jacket-Winter-Thermal-Coats-with-Pockets_7711d647-ab8f-4ba4-a59e-c79c3677871d.e0a0ec368f0bb23d83210d59bee80b50.jpeg"', 'product_url': 'https://www.walmart.com/ip/Men-Casual-Sherpa-Fleece-

In [6]:
def workflow(query):
    refined_query = refine(query)
    print(refined_query)
    result = hybrid_searcher.search(refined_query)
    return result

In [9]:
print (workflow("cheap jacket for men"))

{"semantic_query": "men's jacket", "filters": { "max_price": 70, "category": "men's jacket" }, "financial_priority": "low_total_price"}
[{'actual_price': 100.0, 'discounted_price': 35.99, 'category': 'Block Heels', 'rating': 4.4, 'image_url': '"https://i5.walmartimages.com/seo/Journee-Collection-Womens-Metallic-Rhinestone-Slingbacks_dd6e31f6-b4f4-4a29-91a8-fa10dad11ee5.05590748aba0c01d8d2bc8670fa6c70a.jpeg"', 'product_url': 'https://www.walmart.com/ip/Journee-Collection-Womens-Metallic-Rhinestone-Slingbacks/1597065973'}, {'actual_price': 11.89, 'discounted_price': 10.46, 'category': 'Womens Sweaters', 'rating': 1.0, 'image_url': '"https://i5.walmartimages.com/asr/a1210f14-9d2e-451c-80f3-e426186db6f6_1.b5478189f97475c084900d8528ebee48.jpeg"', 'product_url': 'https://www.walmart.com/ip/Women-Hooded-Sweater-Coat-Winter-Warm-Wool-Zipper-Coat-Cotton-Coat-Outwearpink-ladies-jacket-Clearance/333588477'}, {'actual_price': 40.98, 'discounted_price': 28.68, 'category': 'Black Scrubs', 'rating': 